In [1]:
#Imports
import numpy as np
from load_hyperparams import load_hyperparams
from predict_satellites import get_satellite_properties
from predict_orphans import get_orphan_satellite_properties
from fix_lmc_coords import get_lmc_coords, rotate_about_LMC
from astropy.coordinates import SkyCoord
from masks import load_mask, evaluate_mask
from ssf import load_ssf, apply_ssfs

In [2]:
#Set parameters
params = {}

params['alpha'] = -1.44
params['sigma_M'] = 0.2
params['M50'] = 7.5
params['sigma_mpeak'] = 0.1
params['B'] = 1.
params['A'] = 30
params['sigma_r'] = 0.3
params['n'] = 1.

#Load hyperparameters
hparams, cosmo_params, orphan_params, halo_data, vpeak_Mr_interp = load_hyperparams()

In [3]:
#Load survey masks and ssfs
surveys = ['des', 'ps1']

masks = {}
ssfs = {}

print('loading masks and ssfs ...')
for survey in surveys:
    masks[survey] = load_mask('{}'.format(survey))
    ssfs[survey] = load_ssf(survey)
print('done')

loading masks and ssfs ...
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
NSIDE = 128
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


/Users/eon/Documents/subhalo_satellite_connection/1912.03303/ssf.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.config = yaml.load(open(config_file))


NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
NSIDE = 128
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
done


In [4]:
#Return satellite properties for a particular MW-like host halo
satellite_properties = get_satellite_properties(halo_data[14], params, hparams, cosmo_params, vpeak_Mr_interp)
print(satellite_properties)

{'Mr': array([-1.84030863e+01, -1.73703789e+01, -1.21682804e+01, -1.13109718e+01,
       -1.42000350e+01, -1.50886252e+01, -8.72091511e+00, -9.42592430e+00,
       -1.01610182e+01, -9.20984012e+00, -6.91643329e+00, -8.85455449e+00,
       -7.14663763e+00, -8.00063938e+00, -1.18500563e+01, -8.62890377e+00,
       -7.82005175e+00, -8.97118620e+00, -6.10575381e+00, -9.55106008e+00,
       -8.44721440e+00, -1.00840536e+01, -5.62314809e+00, -5.99475926e+00,
       -9.52832017e+00, -5.54247386e+00, -5.74797029e+00, -4.08230740e+00,
       -6.30056320e+00, -7.18102459e+00, -3.56915297e+00, -8.13110393e+00,
       -4.77342440e+00, -4.88130964e+00, -7.29719409e+00, -5.20092710e+00,
       -9.10014337e+00, -1.00985388e+01, -1.01388141e+01, -4.59471899e+00,
       -4.31215554e+00, -3.45672320e+00, -7.28156091e+00, -3.43218304e+00,
       -7.23394408e+00, -6.30802829e+00, -6.95939392e+00, -3.55351348e+00,
       -7.48253559e+00, -3.96823172e+00, -2.10077724e+00, -5.79601400e+00,
       -5.52312281

In [5]:
#Return orphan satellite properties for a particular host halo
orphan_satellite_properties = get_orphan_satellite_properties(halo_data[14], params, hparams, 
                                                              cosmo_params, vpeak_Mr_interp)
print(orphan_satellite_properties)

{'Mr': array([ -3.73914996,  -7.02851411,  -6.3993008 ,  -7.07806177,
        -1.82473401,   1.51343652,  -1.59120886,  -0.83786946,
         1.80609284,   1.81418328,  -0.87228092,   1.39496636,
         2.02398813,   1.62660992,   0.7797166 ,   1.5725031 ,
         1.29933002,   1.5119617 ,  -1.14601637,   1.16118841,
         1.71068172,   1.78449845,  -0.78656408,  -0.37585344,
         0.51109754,   1.94182942,   1.29015464,   1.83990029,
         1.73025425,   1.53402333,   2.24281258,   0.9426371 ,
         0.68158658,   1.80025488,  -1.0478761 ,   1.29316515,
        -0.55837084,   1.61815229,   1.20388964,   1.78027184,
         1.5823899 ,  -3.28054682,   1.10897466,   0.22983871,
         0.16907471,   0.93917308,   2.09597892,  -1.65297715,
         1.39002912,  -0.96359571,   2.3701872 ,   2.0519996 ,
         1.11511132,  -0.69319356,   1.99531991,   0.86259502,
         1.58027479, -15.70396603,   0.92827165,  -0.14345831,
         0.93841037,   1.59033757,  -0.46792454,

In [6]:
#Rotate about LMC
lmc_ind = 0
lmc_cartesian_coords = get_lmc_coords(halo_data[14],cosmo_params,lmc_ind)
satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[0],cosmo_params,
                                                lmc_cartesian_coords,lmc_ind,observer_ind=0)

In [7]:
#Precompute satellite properties and flags for each observer location
satellite_properties_list = []
DES_flags_list = []
PS1_flags_list = []
Halo_ra_list = []
Halo_dec_list = []

#Loop over simulations
for ind in [14, 20]:
    #Loop over observer locations
    for i in range(6):
        #Get satellite properties
        satellite_properties = get_satellite_properties(halo_data[ind], params, hparams, cosmo_params, vpeak_Mr_interp)
        satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[ind],cosmo_params,
                                                    lmc_cartesian_coords,lmc_ind,i)
        #Transform to sky coordinates
        Halo_sky_coord = SkyCoord(x=satellite_properties_rotated['rotated_pos'][:,0], 
                          y=satellite_properties_rotated['rotated_pos'][:,1], 
                          z=satellite_properties_rotated['rotated_pos'][:,2], 
                          unit='kpc', representation_type='cartesian').spherical
        satellite_properties_rotated['ra'] = Halo_sky_coord.lon.degree
        satellite_properties_rotated['dec'] = Halo_sky_coord.lat.degree
        #Assign flags 
        for survey in surveys:
            satellite_properties_rotated['{}_flags'.format(survey)] = evaluate_mask(Halo_sky_coord.lon.degree,
                                                                                    Halo_sky_coord.lat.degree,
                                                                                    masks[survey], survey)
        #Apply ssfs
        satellite_properties_rotated['pdet'] = apply_ssfs(satellite_properties_rotated,ssfs)
        #Append
        satellite_properties_list.append(satellite_properties_rotated)

In [8]:
#First look at predictions
des_sum = []
ps1_sum = []

for i in range(len(satellite_properties_list)):
    des_flags = satellite_properties_list[i]['des_flags']
    ps1_flags = np.logical_and(satellite_properties_list[i]['ps1_flags'],~satellite_properties_list[i]['des_flags'])
    des_sum.append(np.sum(satellite_properties_list[i]['pdet'][des_flags]))
    ps1_sum.append(np.sum(satellite_properties_list[i]['pdet'][ps1_flags]))
    
print('number of predicted des satellites = {}'.format(np.mean(des_sum,axis=0)))
print('number of predicted ps1 satellites = {}'.format(np.mean(ps1_sum,axis=0)))

number of predicted des satellites = 17.239316077583847
number of predicted ps1 satellites = 25.325583635202705
